# ComfyUI - Workflows Video via API

**Module :** 03-Video-Orchestration  
**Niveau :** Avance  
**Technologies :** ComfyUI API, AnimateDiff nodes, VideoHelperSuite  
**Duree estimee :** 60 minutes  
**VRAM :** ~20 GB (depends on workflow)  

## Objectifs d'Apprentissage

- [ ] Se connecter a l'API ComfyUI et verifier l'etat du serveur
- [ ] Soumettre un workflow AnimateDiff via l'API JSON
- [ ] Utiliser les nodes VideoHelperSuite pour l'entree/sortie de frames
- [ ] Suivre la progression et le monitoring d'un workflow
- [ ] Telecharger et afficher la video generee
- [ ] Comparer l'approche diffusers vs l'approche ComfyUI

## Prerequis

- Instance ComfyUI accessible (locale ou distante)
- Notebooks 03-1 et 03-2 completes
- Custom nodes : AnimateDiff-Evolved, VideoHelperSuite
- Packages : `requests`, `websocket-client`, `torch`, `Pillow`, `imageio`

**Navigation :** [<< 03-2](03-2-Video-Workflow-Orchestration.ipynb) | [Index](../README.md) | [Suivant >>](../04-Applications/04-1-Educational-Video-Generation.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres ComfyUI
comfyui_url = "http://localhost:8188"  # URL du serveur ComfyUI
workflow_type = "animatediff"         # Type de workflow : "animatediff"
enable_video_helper = True           # Utiliser VideoHelperSuite

# Parametres generation
prompt_text = "a serene lake at sunset, soft ripples on water, golden light, cinematic"  # Prompt
negative_prompt = "low quality, blurry, distorted, watermark"  # Prompt negatif
num_frames = 16                      # Nombre de frames
width = 512                          # Largeur
height = 512                         # Hauteur
steps = 25                           # Etapes de debruitage
cfg_scale = 7.5                      # CFG scale
seed_value = 42                      # Graine
fps_output = 8                       # FPS de sortie

# Configuration
run_workflow = True                   # Executer le workflow
save_as_mp4 = True                   # Sauvegarder en MP4
save_results = True

In [2]:
# Parameters
notebook_mode = "batch"
skip_widgets = True


In [3]:
# Setup environnement et imports
import os
import sys
import json
import time
import uuid
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import logging

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging
        print("Helpers GenAI importes")
    except ImportError:
        print("Helpers GenAI non disponibles - mode autonome")

OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'comfyui_video'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('comfyui_video')

print(f"ComfyUI - Workflows Video via API")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"ComfyUI URL : {comfyui_url}")
print(f"Workflow : {workflow_type}")
print(f"VideoHelperSuite : {enable_video_helper}")

Helpers GenAI importes
ComfyUI - Workflows Video via API
Date : 2026-02-26 08:10:11
Mode : batch
ComfyUI URL : http://localhost:8188
Workflow : animatediff
VideoHelperSuite : True


In [4]:
# Chargement .env et verification de la connexion ComfyUI
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve")

# Token d'authentification ComfyUI
comfyui_token = os.environ.get('COMFYUI_AUTH_TOKEN', os.environ.get('COMFYUI_BEARER_TOKEN', ''))
if comfyui_token:
    print(f"Token ComfyUI : configure ({comfyui_token[:8]}...)")
else:
    print("Token ComfyUI : non configure (connexion sans authentification)")

# Verification de la connexion
print("\n--- VERIFICATION COMFYUI ---")
print("=" * 40)

import requests

comfyui_available = False
comfyui_info = {}

headers = {}
if comfyui_token:
    headers['Authorization'] = f'Bearer {comfyui_token}'

try:
    # Health check
    response = requests.get(f"{comfyui_url}/system_stats", headers=headers, timeout=10)
    if response.status_code == 200:
        stats = response.json()
        comfyui_available = True
        comfyui_info = stats
        
        print(f"ComfyUI accessible : {comfyui_url}")
        
        # GPU info
        devices = stats.get('devices', [])
        for dev in devices:
            name = dev.get('name', 'Inconnu')
            vram_total = dev.get('vram_total', 0) / 1024**3
            vram_free = dev.get('vram_free', 0) / 1024**3
            print(f"  GPU : {name}")
            print(f"  VRAM : {vram_free:.1f} / {vram_total:.1f} GB libre")
        
        # Verifier les custom nodes
        try:
            obj_info = requests.get(f"{comfyui_url}/object_info", headers=headers, timeout=30)
            if obj_info.status_code == 200:
                nodes = obj_info.json()
                animatediff_nodes = [n for n in nodes.keys() if 'animatediff' in n.lower()]
                vhs_nodes = [n for n in nodes.keys() if 'video' in n.lower() and 'helper' in n.lower()]
                
                print(f"\n  AnimateDiff nodes : {len(animatediff_nodes)}")
                for n in animatediff_nodes[:5]:
                    print(f"    - {n}")
                if len(animatediff_nodes) > 5:
                    print(f"    ... et {len(animatediff_nodes) - 5} autres")
                
                print(f"  VideoHelperSuite nodes : {len(vhs_nodes)}")
                for n in vhs_nodes[:5]:
                    print(f"    - {n}")
                
                if not animatediff_nodes:
                    print("\n  AnimateDiff-Evolved non installe dans ComfyUI")
                    print("  Le notebook montrera les workflows sans les executer")
        except Exception:
            print("  Impossible de lister les nodes")
    else:
        print(f"ComfyUI repond avec status {response.status_code}")
        run_workflow = False

except requests.ConnectionError:
    print(f"ComfyUI non accessible a {comfyui_url}")
    print("Le notebook montrera les workflows sans les executer.")
    run_workflow = False

except Exception as e:
    print(f"Erreur connexion : {type(e).__name__}: {str(e)[:100]}")
    run_workflow = False

print(f"\nComfyUI disponible : {comfyui_available}")
print(f"Workflow active : {run_workflow}")

Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env
Token ComfyUI : non configure (connexion sans authentification)

--- VERIFICATION COMFYUI ---


ComfyUI non accessible a http://localhost:8188
Le notebook montrera les workflows sans les executer.

ComfyUI disponible : False
Workflow active : False


## Section 1 : API ComfyUI et soumission de workflows

ComfyUI expose une API REST et WebSocket pour piloter la generation.
Contrairement a l'approche diffusers (Python direct), ComfyUI utilise
des workflows JSON (graphes de nodes) que l'on soumet a un serveur.

| Endpoint | Methode | Description |
|----------|---------|-------------|
| `/system_stats` | GET | Etat du serveur (GPU, queue) |
| `/object_info` | GET | Liste des nodes disponibles |
| `/prompt` | POST | Soumettre un workflow |
| `/history/{id}` | GET | Resultats d'un workflow execute |
| `/view` | GET | Telecharger une image/video generee |
| `/ws` | WebSocket | Suivi en temps reel de la progression |

### Architecture ComfyUI vs diffusers

| Aspect | diffusers (Python) | ComfyUI (API) |
|--------|--------------------|---------------|
| Flexibilite | Code personnalise | Workflow visuel/JSON |
| Reproductibilite | Script Python | Fichier JSON exportable |
| GPU requis localement | Oui | Non (serveur distant possible) |
| Custom nodes | Non applicable | Ecosysteme riche |
| Monitoring | Manuel | WebSocket integre |

In [5]:
# Definition du workflow AnimateDiff via l'API ComfyUI
print("\n--- WORKFLOW ANIMATEDIFF ---")
print("=" * 45)


def build_animatediff_workflow(
    prompt: str,
    negative: str,
    width: int,
    height: int,
    frames: int,
    steps: int,
    cfg: float,
    seed: int
) -> Dict[str, Any]:
    """
    Construit un workflow AnimateDiff au format ComfyUI API.
    
    Le workflow suit la chaine :
    CheckpointLoader -> CLIPTextEncode (pos/neg) -> AnimateDiffLoader
    -> KSampler -> VAEDecode -> VHS_VideoCombine (ou SaveImage)
    
    Args:
        prompt: Description textuelle positive
        negative: Prompt negatif
        width, height: Resolution video
        frames: Nombre de frames
        steps: Etapes de debruitage
        cfg: CFG scale
        seed: Graine de generation
    
    Returns:
        Dict au format ComfyUI prompt API
    """
    workflow = {
        # Node 1 : Charger le modele SD 1.5
        "1": {
            "class_type": "CheckpointLoaderSimple",
            "inputs": {
                "ckpt_name": "v1-5-pruned-emaonly.safetensors"
            }
        },
        # Node 2 : Encodage du prompt positif
        "2": {
            "class_type": "CLIPTextEncode",
            "inputs": {
                "text": prompt,
                "clip": ["1", 1]  # Sortie CLIP du checkpoint
            }
        },
        # Node 3 : Encodage du prompt negatif
        "3": {
            "class_type": "CLIPTextEncode",
            "inputs": {
                "text": negative,
                "clip": ["1", 1]
            }
        },
        # Node 4 : Charger le motion module AnimateDiff
        "4": {
            "class_type": "ADE_LoadAnimateDiffModel",
            "inputs": {
                "model_name": "v3_sd15_mm.ckpt"
            }
        },
        # Node 5 : Appliquer AnimateDiff au modele
        "5": {
            "class_type": "ADE_ApplyAnimateDiffModelSimple",
            "inputs": {
                "model": ["1", 0],  # Sortie MODEL du checkpoint
                "motion_model": ["4", 0]
            }
        },
        # Node 6 : Latent vide (batch de frames)
        "6": {
            "class_type": "EmptyLatentImage",
            "inputs": {
                "width": width,
                "height": height,
                "batch_size": frames
            }
        },
        # Node 7 : KSampler
        "7": {
            "class_type": "KSampler",
            "inputs": {
                "model": ["5", 0],  # Modele avec AnimateDiff
                "positive": ["2", 0],
                "negative": ["3", 0],
                "latent_image": ["6", 0],
                "seed": seed,
                "steps": steps,
                "cfg": cfg,
                "sampler_name": "euler",
                "scheduler": "normal",
                "denoise": 1.0
            }
        },
        # Node 8 : Decodage VAE
        "8": {
            "class_type": "VAEDecode",
            "inputs": {
                "samples": ["7", 0],
                "vae": ["1", 2]  # Sortie VAE du checkpoint
            }
        }
    }
    
    # Node 9 : Sortie video (VideoHelperSuite ou SaveImage)
    if enable_video_helper:
        workflow["9"] = {
            "class_type": "VHS_VideoCombine",
            "inputs": {
                "images": ["8", 0],
                "frame_rate": fps_output,
                "loop_count": 0,
                "filename_prefix": "animatediff",
                "format": "video/h264-mp4",
                "pingpong": False,
                "save_output": True
            }
        }
    else:
        workflow["9"] = {
            "class_type": "SaveImage",
            "inputs": {
                "images": ["8", 0],
                "filename_prefix": "animatediff_frame"
            }
        }
    
    return workflow


# Construire le workflow
workflow = build_animatediff_workflow(
    prompt=prompt_text,
    negative=negative_prompt,
    width=width,
    height=height,
    frames=num_frames,
    steps=steps,
    cfg=cfg_scale,
    seed=seed_value
)

print(f"Workflow construit : {len(workflow)} nodes")
print(f"\nNodes du workflow :")
for node_id, node_data in workflow.items():
    print(f"  [{node_id}] {node_data['class_type']}")

# Sauvegarder le workflow JSON
workflow_path = OUTPUT_DIR / "animatediff_workflow.json"
with open(workflow_path, 'w', encoding='utf-8') as f:
    json.dump({"prompt": workflow}, f, indent=2)
print(f"\nWorkflow sauvegarde : {workflow_path.name}")


--- WORKFLOW ANIMATEDIFF ---
Workflow construit : 9 nodes

Nodes du workflow :
  [1] CheckpointLoaderSimple
  [2] CLIPTextEncode
  [3] CLIPTextEncode
  [4] ADE_LoadAnimateDiffModel
  [5] ADE_ApplyAnimateDiffModelSimple
  [6] EmptyLatentImage
  [7] KSampler
  [8] VAEDecode
  [9] VHS_VideoCombine

Workflow sauvegarde : animatediff_workflow.json


### Interpretation : Structure du workflow

| Node | Classe | Role |
|------|--------|------|
| 1 | `CheckpointLoaderSimple` | Charge le modele SD 1.5 (MODEL + CLIP + VAE) |
| 2-3 | `CLIPTextEncode` | Encode les prompts positif et negatif |
| 4 | `ADE_LoadAnimateDiffModel` | Charge le motion module v3 |
| 5 | `ADE_ApplyAnimateDiffModelSimple` | Applique le motion module au modele |
| 6 | `EmptyLatentImage` | Cree un batch de latents (1 par frame) |
| 7 | `KSampler` | Debruite les latents |
| 8 | `VAEDecode` | Decode les latents en images |
| 9 | `VHS_VideoCombine` | Assemble les frames en video MP4 |

**Points cles** :
1. Chaque node est identifie par un ID unique (string) et a un `class_type`
2. Les connexions sont exprimees par `["node_id", output_index]`
3. Le workflow est un DAG (graphe acyclique dirige) - pas de boucles

## Section 2 : Soumission et monitoring du workflow

In [6]:
# Soumission du workflow et monitoring
print("\n--- SOUMISSION DU WORKFLOW ---")
print("=" * 45)

client_id = str(uuid.uuid4())
prompt_id = None
generation_result = {"success": False}


def submit_workflow(workflow: Dict, server_url: str, auth_headers: Dict) -> Optional[str]:
    """Soumet un workflow a ComfyUI et retourne le prompt_id."""
    payload = {
        "prompt": workflow,
        "client_id": client_id
    }
    
    try:
        response = requests.post(
            f"{server_url}/prompt",
            json=payload,
            headers=auth_headers,
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            return result.get('prompt_id')
        else:
            print(f"  Erreur HTTP {response.status_code} : {response.text[:200]}")
            return None
    except Exception as e:
        print(f"  Erreur soumission : {type(e).__name__}: {str(e)[:100]}")
        return None


def poll_progress(prompt_id: str, server_url: str, auth_headers: Dict,
                  timeout: int = 300, poll_interval: float = 2.0) -> Dict:
    """
    Attend la fin d'un workflow en interrogeant periodiquement l'historique.
    
    Retourne le resultat du workflow ou un dict d'erreur.
    """
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        try:
            response = requests.get(
                f"{server_url}/history/{prompt_id}",
                headers=auth_headers,
                timeout=10
            )
            
            if response.status_code == 200:
                history = response.json()
                if prompt_id in history:
                    entry = history[prompt_id]
                    status = entry.get('status', {})
                    
                    if status.get('completed', False):
                        elapsed = time.time() - start_time
                        return {
                            "success": True,
                            "outputs": entry.get('outputs', {}),
                            "elapsed": elapsed
                        }
                    
                    if status.get('status_str') == 'error':
                        return {
                            "success": False,
                            "error": "Workflow en erreur",
                            "details": status
                        }
        except Exception:
            pass
        
        elapsed = time.time() - start_time
        print(f"  En cours... {elapsed:.0f}s", end='\r')
        time.sleep(poll_interval)
    
    return {"success": False, "error": f"Timeout apres {timeout}s"}


def download_outputs(outputs: Dict, server_url: str, auth_headers: Dict) -> List[Image.Image]:
    """
    Telecharge les images/videos generees depuis les outputs du workflow.
    """
    images = []
    
    for node_id, node_output in outputs.items():
        # Images standard (SaveImage node)
        if 'images' in node_output:
            for img_info in node_output['images']:
                filename = img_info.get('filename', '')
                subfolder = img_info.get('subfolder', '')
                img_type = img_info.get('type', 'output')
                
                params = {
                    'filename': filename,
                    'subfolder': subfolder,
                    'type': img_type
                }
                
                try:
                    resp = requests.get(
                        f"{server_url}/view",
                        params=params,
                        headers=auth_headers,
                        timeout=30
                    )
                    if resp.status_code == 200:
                        from io import BytesIO
                        img = Image.open(BytesIO(resp.content)).convert('RGB')
                        images.append(img)
                except Exception as e:
                    print(f"  Erreur telechargement {filename}: {e}")
        
        # Videos (VHS_VideoCombine node)
        if 'gifs' in node_output:
            for vid_info in node_output['gifs']:
                filename = vid_info.get('filename', '')
                subfolder = vid_info.get('subfolder', '')
                print(f"  Video generee : {filename}")
    
    return images


# Soumettre le workflow
if run_workflow and comfyui_available:
    print(f"Client ID : {client_id[:8]}...")
    print(f"Prompt : {prompt_text[:50]}...")
    print(f"Parametres : {num_frames} frames, {steps} steps, CFG={cfg_scale}")
    
    prompt_id = submit_workflow(workflow, comfyui_url, headers)
    
    if prompt_id:
        print(f"Workflow soumis : {prompt_id[:12]}...")
        print(f"\nAttente de la completion...")
        
        generation_result = poll_progress(prompt_id, comfyui_url, headers, timeout=300)
        
        if generation_result['success']:
            print(f"\nWorkflow termine en {generation_result['elapsed']:.1f}s")
            
            # Telecharger les resultats
            output_images = download_outputs(
                generation_result['outputs'], comfyui_url, headers
            )
            generation_result['images'] = output_images
            
            if output_images:
                print(f"Images telechargees : {len(output_images)}")
                
                # Affichage
                n_display = min(8, len(output_images))
                fig, axes = plt.subplots(2, 4, figsize=(16, 8))
                axes_flat = axes.flatten()
                for i in range(n_display):
                    axes_flat[i].imshow(output_images[i])
                    axes_flat[i].set_title(f"Frame {i+1}/{len(output_images)}", fontsize=9)
                    axes_flat[i].axis('off')
                for i in range(n_display, len(axes_flat)):
                    axes_flat[i].axis('off')
                plt.suptitle(f"ComfyUI AnimateDiff : {prompt_text[:50]}...", fontsize=11, fontweight='bold')
                plt.tight_layout()
                plt.show()
                
                # Sauvegarder en MP4 local
                if save_as_mp4 and output_images:
                    import imageio
                    mp4_path = OUTPUT_DIR / "comfyui_animatediff.mp4"
                    frames_array = [np.array(img) for img in output_images]
                    imageio.mimsave(str(mp4_path), frames_array, fps=fps_output)
                    print(f"MP4 sauvegarde : {mp4_path.name}")
            else:
                print("Aucune image telechargee (la sortie est peut-etre une video)")
        else:
            print(f"\nErreur : {generation_result.get('error', 'inconnue')}")
    else:
        print("Soumission echouee")
else:
    print("Workflow non execute (ComfyUI non disponible ou desactive)")
    print("\nSchema d'execution :")
    print("  1. POST /prompt avec le workflow JSON")
    print("  2. Poll /history/{prompt_id} pour attendre la completion")
    print("  3. GET /view pour telecharger les images/videos")


--- SOUMISSION DU WORKFLOW ---
Workflow non execute (ComfyUI non disponible ou desactive)

Schema d'execution :
  1. POST /prompt avec le workflow JSON
  2. Poll /history/{prompt_id} pour attendre la completion
  3. GET /view pour telecharger les images/videos


### Interpretation : Execution du workflow

| Etape | Endpoint | Description |
|-------|----------|-------------|
| Soumission | `POST /prompt` | Envoie le workflow, retourne `prompt_id` |
| Monitoring | `GET /history/{id}` | Interroge periodiquement l'etat |
| Telechargement | `GET /view` | Recupere les images/videos generees |

**Points cles** :
1. ComfyUI traite les workflows de facon asynchrone (queue)
2. Le monitoring par polling est simple mais moins reactif que WebSocket
3. Les images sont stockees sur le serveur et telechargees a la demande

## Section 3 : Comparaison diffusers vs ComfyUI

Nous avons explore les deux approches de generation video. Comparons-les objectivement.

In [7]:
# Comparaison diffusers vs ComfyUI
print("\n--- COMPARAISON DIFFUSERS VS COMFYUI ---")
print("=" * 50)

comparison_data = {
    "Critere": [
        "Installation",
        "Flexibilite code",
        "Interface visuelle",
        "Custom nodes",
        "Reproductibilite",
        "GPU distant",
        "Debugging",
        "Integration Python",
        "Community",
        "Production batch"
    ],
    "diffusers": [
        "pip install",
        "Totale (Python)",
        "Non",
        "Non applicable",
        "Script Python",
        "Non (local only)",
        "pdb / print",
        "Native",
        "HuggingFace",
        "Scripts Python"
    ],
    "ComfyUI": [
        "Git clone + setup",
        "Via nodes JSON",
        "Oui (navigateur)",
        "Ecosysteme riche",
        "Workflow JSON",
        "Oui (API REST)",
        "Logs serveur",
        "Via API REST",
        "ComfyUI Manager",
        "API programmable"
    ]
}

# Affichage du tableau
print(f"{'Critere':<22} {'diffusers':<22} {'ComfyUI':<22}")
print("-" * 66)
for i in range(len(comparison_data['Critere'])):
    print(f"  {comparison_data['Critere'][i]:<22} {comparison_data['diffusers'][i]:<22} {comparison_data['ComfyUI'][i]:<22}")

# Recapitulatif du notebook
print(f"\n\n--- RECAPITULATIF DU MODULE 03 ---")
print("=" * 50)

print(f"\n{'Notebook':<45} {'Contenu principal':<40}")
print("-" * 85)
print(f"  {'03-1 Multi-Model Comparison':<45} {'Benchmark HunyuanVideo/LTX/Wan/SVD':<40}")
print(f"  {'03-2 Workflow Orchestration':<45} {'Pipelines multi-modeles, batch, upscale':<40}")
print(f"  {'03-3 ComfyUI Video Workflows':<45} {'API ComfyUI, AnimateDiff, monitoring':<40}")

print(f"\nRecommandation selon le profil :")
print(f"  Chercheur/Dev Python : diffusers (controle total, scripts personnalises)")
print(f"  Artiste/Designer : ComfyUI (interface visuelle, workflows partageables)")
print(f"  Production : ComfyUI API (GPU distant, file d'attente, monitoring)")


--- COMPARAISON DIFFUSERS VS COMFYUI ---
Critere                diffusers              ComfyUI               
------------------------------------------------------------------
  Installation           pip install            Git clone + setup     
  Flexibilite code       Totale (Python)        Via nodes JSON        
  Interface visuelle     Non                    Oui (navigateur)      
  Custom nodes           Non applicable         Ecosysteme riche      
  Reproductibilite       Script Python          Workflow JSON         
  GPU distant            Non (local only)       Oui (API REST)        
  Debugging              pdb / print            Logs serveur          
  Integration Python     Native                 Via API REST          
  Community              HuggingFace            ComfyUI Manager       
  Production batch       Scripts Python         API programmable      


--- RECAPITULATIF DU MODULE 03 ---

Notebook                                      Contenu principal           

In [8]:
# Mode interactif : modifier les parametres du workflow
if notebook_mode == "interactive" and not skip_widgets:
    print("\n--- MODE INTERACTIF ---")
    print("=" * 40)
    print("Entrez un prompt pour generer via ComfyUI AnimateDiff.")
    print("(Laissez vide pour passer a la suite)")
    
    try:
        user_prompt = input("\nVotre prompt : ").strip()
        
        if user_prompt and run_workflow and comfyui_available:
            print(f"\nGeneration : {user_prompt}")
            
            user_workflow = build_animatediff_workflow(
                prompt=user_prompt,
                negative=negative_prompt,
                width=width,
                height=height,
                frames=num_frames,
                steps=steps,
                cfg=cfg_scale,
                seed=seed_value + 100
            )
            
            user_prompt_id = submit_workflow(user_workflow, comfyui_url, headers)
            if user_prompt_id:
                print(f"Workflow soumis. Attente...")
                user_result = poll_progress(user_prompt_id, comfyui_url, headers)
                
                if user_result['success']:
                    user_images = download_outputs(user_result['outputs'], comfyui_url, headers)
                    if user_images:
                        n_display = min(8, len(user_images))
                        fig, axes = plt.subplots(1, n_display, figsize=(2.5 * n_display, 3))
                        if n_display == 1:
                            axes = [axes]
                        for i in range(n_display):
                            axes[i].imshow(user_images[i])
                            axes[i].set_title(f"Frame {i+1}", fontsize=8)
                            axes[i].axis('off')
                        plt.suptitle(f"ComfyUI : {user_prompt[:50]}...", fontweight='bold')
                        plt.tight_layout()
                        plt.show()
                    print(f"Generation terminee en {user_result['elapsed']:.1f}s")
                else:
                    print(f"Erreur : {user_result.get('error', 'inconnue')}")
        elif user_prompt:
            print("ComfyUI non disponible")
        else:
            print("Mode interactif ignore")
    
    except (KeyboardInterrupt, EOFError) as e:
        print(f"\nMode interactif interrompu ({type(e).__name__})")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("\nMode interactif non disponible (execution automatisee)")
        else:
            print(f"\nErreur inattendue : {error_type} - {str(e)[:100]}")
            print("Passage a la suite du notebook")
else:
    print("\nMode batch - Interface interactive desactivee")


Mode batch - Interface interactive desactivee


## Bonnes pratiques ComfyUI pour la video

### Workflow AnimateDiff - conseils

| Conseil | Details |
|---------|--------|
| Modele de base | SD 1.5 (recommande pour AnimateDiff v3) |
| Motion module | `v3_sd15_mm.ckpt` (meilleure coherence) |
| Sampler | `euler` ou `euler_ancestral` |
| CFG | 6.0-8.0 (eviter > 10 pour la coherence temporelle) |
| Frames | 16-24 (au-dela, la coherence diminue) |
| Resolution | 512x512 (limitation SD 1.5) |

### VideoHelperSuite - nodes essentiels

| Node | Usage |
|------|-------|
| `VHS_VideoCombine` | Assembler les frames en video MP4/GIF |
| `VHS_LoadVideo` | Charger une video existante comme input |
| `VHS_SplitVideo` | Decomposer une video en frames |
| `VHS_MergeImages` | Combiner des sequences de frames |

### Securite et authentification

| Aspect | Recommandation |
|--------|---------------|
| Token Bearer | Toujours utiliser en production |
| HTTPS | Obligatoire pour les connexions distantes |
| Timeouts | Configurer des timeouts raisonnables (5 min max) |
| Validation | Verifier les outputs avant traitement |

In [9]:
# Statistiques de session et prochaines etapes
print("\n--- STATISTIQUES DE SESSION ---")
print("=" * 40)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"ComfyUI URL : {comfyui_url}")
print(f"ComfyUI disponible : {comfyui_available}")
print(f"Workflow type : {workflow_type}")
print(f"VideoHelperSuite : {enable_video_helper}")
print(f"Parametres : {num_frames} frames, {steps} steps, CFG={cfg_scale}")
print(f"Resolution : {width}x{height}")

if generation_result.get('success'):
    print(f"\nGeneration : OK en {generation_result.get('elapsed', 0):.1f}s")
    n_images = len(generation_result.get('images', []))
    print(f"Images telechargees : {n_images}")
else:
    print(f"\nGeneration : non executee ou echouee")

if save_results and OUTPUT_DIR.exists():
    generated_files = list(OUTPUT_DIR.glob('*'))
    print(f"\nFichiers generes ({len(generated_files)}) :")
    for f in sorted(generated_files):
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name} ({size_kb:.1f} KB)")

print(f"\n--- PROCHAINES ETAPES ---")
print(f"1. Module 04-1 : Generation video educative (applications production)")
print(f"2. Explorer les workflows ComfyUI avances (ControlNet video, IP-Adapter)")
print(f"3. Combiner ComfyUI + diffusers dans un pipeline hybride")
print(f"4. Deployer un serveur ComfyUI pour la generation a la demande")

print(f"\nNotebook 03-3 ComfyUI Video Workflows termine - {datetime.now().strftime('%H:%M:%S')}")


--- STATISTIQUES DE SESSION ---
Date : 2026-02-26 08:10:16
Mode : batch
ComfyUI URL : http://localhost:8188
ComfyUI disponible : False
Workflow type : animatediff
VideoHelperSuite : True
Parametres : 16 frames, 25 steps, CFG=7.5
Resolution : 512x512

Generation : non executee ou echouee

Fichiers generes (1) :
  animatediff_workflow.json (2.1 KB)

--- PROCHAINES ETAPES ---
1. Module 04-1 : Generation video educative (applications production)
2. Explorer les workflows ComfyUI avances (ControlNet video, IP-Adapter)
3. Combiner ComfyUI + diffusers dans un pipeline hybride
4. Deployer un serveur ComfyUI pour la generation a la demande

Notebook 03-3 ComfyUI Video Workflows termine - 08:10:16
